# Dataset Creation Pipeline

### Imports 

In [ ]:
import os

from dataset_creation.sheets_generator import RandomMusicGenerator
from conversion_tools import convert_mxls_to_svg, convert_svgs_to_png
from image_pair_tools import get_files_clean_dirty, create_dict_clean_dirty, convert_dict_clean_dirty_to_df

### Configuration

In [4]:
# Set variables and paths
dataset_path: str = '../dataset/'
num_generated_mxls: int = 2

# Artificial Music Sheets
sheets_generation_path: [None, str] = os.path.join(dataset_path, "generated")
sheets_generation_xml_path: str = os.path.join(sheets_generation_path, "musicxml")
sheets_generation_svg_path: str = os.path.join(sheets_generation_path, "render_svg")
sheets_generation_png_path: str = os.path.join(sheets_generation_path, "render_png")


# Existing music sheets
existing_sheets_path: [None, str] = os.path.join(dataset_path, "existing")
existing_sheets_mxl_path: str = os.path.join(existing_sheets_path, "musicxml")
existing_sheets_svg_path: str = os.path.join(existing_sheets_path, "render_svg")
existing_sheets_png_path: str = os.path.join(existing_sheets_path, "render_png")

pairs_clean_path = os.path.join(dataset_path, "pairs", "clean")
pairs_dirty_path = os.path.join(dataset_path, "pairs", "dirty")
index_clean_dirty = os.path.join(dataset_path, "pairs", "clean_dirty_index.csv")

# Create derived variables and check (+create) paths 
convert_existing_sheets: bool = existing_sheets_path is not None
create_artificial_sheets: bool = sheets_generation_path is not None

assert num_generated_mxls > 0
assert os.path.isdir(dataset_path)
assert os.path.isdir(sheets_generation_path) or not create_artificial_sheets
assert os.path.isdir(existing_sheets_path) or not convert_existing_sheets

if create_artificial_sheets:
    os.makedirs(sheets_generation_xml_path, exist_ok=True)
    os.makedirs(sheets_generation_svg_path, exist_ok=True)
    os.makedirs(sheets_generation_png_path, exist_ok=True)
    
if convert_existing_sheets:
    assert os.path.isdir(existing_sheets_mxl_path), "Needs existing musicxml files to convert them"
    assert len(os.listdir(existing_sheets_mxl_path)) > 0, "Needs existing musicxml files to convert them"
    os.makedirs(existing_sheets_svg_path, exist_ok=True)
    os.makedirs(existing_sheets_png_path, exist_ok=existing_sheets_path)

## Generate Music Sheets
### Artificial Music Sheets

Here we create n music sheets, save them as MusicXML (MXLs) and convert them to SVGs and PNGs.

In [ ]:
if create_artificial_sheets:
    for i in range(num_generated_mxls):
        rmg: RandomMusicGenerator = RandomMusicGenerator()
        rmg.play_music_to_xml(sheets_generation_xml_path, f"generated_{i}")
       
    convert_mxls_to_svg(sheets_generation_xml_path, sheets_generation_svg_path)
    convert_svgs_to_png(sheets_generation_svg_path, sheets_generation_png_path)

### Existing Music Sheets
Here we convert existing MusicXML (MXLs) files to SVGs and PNGs.

In [ ]:
if convert_existing_sheets:
    convert_mxls_to_svg(existing_sheets_mxl_path, existing_sheets_svg_path)
    convert_svgs_to_png(existing_sheets_svg_path, existing_sheets_png_path)

Now it's time to apply some data augmentations using [DocCreator](https://doc-creator.labri.fr/). 
After the creation we can continue with the next step.

# Create index for identification during training

In [ ]:
files_clean, files_dirty = get_files_clean_dirty(pairs_clean_path, pairs_dirty_path)
dict_clean_dirty = create_dict_clean_dirty(files_clean, files_dirty)
df_clean_dirty = convert_dict_clean_dirty_to_df(dict_clean_dirty)
df_clean_dirty

In [ ]:
df_clean_dirty.to_csv(index_clean_dirty)